In [18]:
import pandas as pd
from cassandra.cluster import Cluster

In [19]:
df = pd.read_csv('./inventory.csv')
df

,sku,name,description,warehouse_num
0,'SK4567A','AMEROUS','15 Inches Magnetic Wooden Chess Set - Beginn...,1433
1,'PH6788G','Taco vs Burrito','The Strategic Family Friendly Card Game Crea...,6677
2,'YU4456Z','Wingspan Board Game','A Bird-Collection Engine-Building STONEMAIER...,8760
3,'NZ6789T','Skillmatics Card Game','Found It Indoor Edition Scavenger Hunt for K...,6677
4,'GY3452I','Azul-Board Game Strategy','Board Game Mosaic-Tile Placement Game Family...,1433


In [20]:
clstr = Cluster()
session = clstr.connect()

In [21]:
session.execute("CREATE KEYSPACE IF NOT EXISTS test WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}")

In [22]:
rows = session.execute("desc keyspaces")

In [23]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

inventory
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
test
w04
w04python


In [24]:
session.execute("""
CREATE TABLE IF NOT EXISTS test.invent ( 
    sku TEXT, 
    name TEXT, 
    description TEXT,  
    warehouse_num INT, 
    PRIMARY KEY(sku)
);
""")

In [25]:
for index, row in df.iterrows():
    session.execute(f"""
        INSERT INTO test.invent (sku, name, description, warehouse_num)     
        VALUES ({row[0].strip()}, {row[1].strip()}, {row[2].strip()}, {row[3]});
        """
       )

In [26]:
session.execute ("CREATE INDEX ON test.invent(warehouse_num)")

InvalidRequest: Error from server: code=2200 [Invalid query] message="Index test is a duplicate of existing index invent_warehouse_num_idx"

In [27]:
rows = session.execute("select (sku, name, description, warehouse_num) from test.invent where warehouse_num = 6677 ALLOW FILTERING")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]},")

sku=PH6788G, name=Taco vs Burrito, description=The Strategic Family Friendly Card Game Created by a 7 Year Old, warehouse_num=6677,
sku=NZ6789T, name=Skillmatics Card Game, description=Found It Indoor Edition Scavenger Hunt for Kids Fun Family Game Ages 4 to 7, warehouse_num=6677,
